In [ ]:
# CREATE TABLE items (id bigserial PRIMARY KEY, embedding vector(1536));
# 1536 important

In [1]:
!pip install psycopg2-binary pgvector

  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/a3/0e/18d7476bc94a26e5adefbdf0cbeec67a15fdfa8d983baacc25d91e1a3330/psycopg2_binary-2.9.7-cp310-cp310-win_amd64.whl.metadata
  Using cached psycopg2_binary-2.9.7-cp310-cp310-win_amd64.whl.metadata (4.6 kB)
Using cached psycopg2_binary-2.9.7-cp310-cp310-win_amd64.whl (1.2 MB)


In [22]:
import psycopg2
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.pgvector import PGVector
from dotenv import load_dotenv

In [24]:
load_dotenv()

loader = TextLoader("./data/state_of_the_union.txt", encoding="utf-8")
documents = loader.load()

print(documents)  # prints the document objects
print(len(documents))  # 1 - we've only read one file/document into the loader

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citize

In [25]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

print(texts)
print(len(texts))

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.', metadata={'source'

In [26]:
print(texts[0])

page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.' metadata={'source': 'state_of

In [27]:
embeddings = OpenAIEmbeddings()

vector = embeddings.embed_query("Testing the embedding model")

print(len(vector))  # 1536 dimensions

1536


In [15]:
# vector

In [28]:
doc_vectors = embeddings.embed_documents([t.page_content for t in texts[:5]])

print(len(doc_vectors))  # 5 vectors in the output
print(doc_vectors[0])  # this will output the first chunk's 1539-dimensional vector

5
[-0.0035596876505259024, -0.010357264458449957, -0.01851170328993628, -0.018020711889863743, 0.005898532008571962, 0.020011216458314753, 0.014928792645285766, -0.009527887244487844, -0.0029957110612126338, -0.006641653850720983, 0.01535343383098843, 0.008950640472602214, -0.020887039660526095, 0.0008190101384859731, 0.001997140707475089, 0.006379570829922891, 0.01829938223142366, -0.014782822732465593, 0.029592184433092042, -0.0224661753213226, 0.00974684304504068, -0.013197053260951453, 0.01539324414587487, 0.012520280701849726, -0.0005730997670721906, 0.011113656716001983, 0.03686416532032681, -0.03407745817943734, 0.019055774488975687, -0.02440360009080674, -0.012414120172593416, -0.02969834589367093, -0.024018768288667936, -0.004750673672502205, -0.024775160546219963, -0.020887039660526095, -0.013349657915811839, -0.01220180004540337, 0.0035099249897484986, -0.018498433805855856, 0.007417951265716229, -0.0023521143077877764, -0.011053941243672325, 0.005696163994442238, -0.0248415

In [29]:
try:
    connection = psycopg2.connect(
        database="postgres",
        user="postgres",
        password="postgres",
        host="host.docker.internal",
    )
    cursor = connection.cursor()

    sql = f"INSERT INTO items (embedding) VALUES ('{vector}');"
    for i in range(len(doc_vectors)):
        sql = f"INSERT INTO items (embedding) VALUES ('{doc_vectors[i]}');"
        cursor.execute(sql)

        connection.commit()
        count = cursor.rowcount
        print(count, "Record inserted successfully into mobile table")


except (Exception, psycopg2.Error) as error:
    print("Failed to insert record into mobile table", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

1 Record inserted successfully into mobile table
1 Record inserted successfully into mobile table
1 Record inserted successfully into mobile table
1 Record inserted successfully into mobile table
1 Record inserted successfully into mobile table
PostgreSQL connection is closed


In [30]:
query_vector = embeddings.embed_query("What di the president say aout Russia?")

In [31]:
query_vector

[-0.015701532673243517,
 0.0009299536069604196,
 0.015309972990193117,
 -0.01012181044940438,
 -0.012790940953968896,
 -0.014109190831406307,
 -0.00813138407843046,
 -0.010885351551955883,
 -0.013508799752012903,
 -0.008405475111507667,
 0.03879048751723788,
 0.008862294431292269,
 -0.007439628855683357,
 -0.022266678950794965,
 0.014056982749489908,
 -0.0010531316718821206,
 0.02436804819433318,
 -0.027905135033959406,
 0.04928427659217559,
 -0.011374800690107588,
 -0.0032842045831279498,
 -0.012751785358192894,
 0.00042786022210617835,
 -0.002099737333524691,
 -0.01840329206485254,
 0.002094842767637367,
 0.024446359385885186,
 -0.007955182500454558,
 0.009312587973667972,
 -0.016589066340531932,
 0.014879257711366714,
 -0.02545136263446939,
 -0.01910810023940134,
 -0.010337168554478878,
 -0.011994770033050292,
 -0.012229705470351494,
 -0.0035175088089075763,
 -0.020439401671656553,
 0.00913638639569207,
 -0.014983673875199511,
 0.016158351993028123,
 0.014944517348100915,
 0.0190428

In [40]:
try:
    connection = psycopg2.connect(
        database="postgres",
        user="postgres",
        password="postgres",
        host="host.docker.internal",
    )
    cursor = connection.cursor()

    sql = """SELECT id, (embedding <=> {query_vector}) as cosine_distance
        FROM items
        ORDER BY cosine_distance
        LIMIT 2;"""
    sql = f"SELECT id FROM items ORDER BY embedding <-> '{query_vector}' LIMIT 2;"
    cursor.execute(sql)
    connection.commit()
    record = cursor.fetchone()
    print(record)

except (Exception, psycopg2.Error) as error:
    print(error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

(5,)
PostgreSQL connection is closed


In [17]:
try:
    connection = psycopg2.connect(
        database="postgres",
        user="postgres",
        password="postgres",
        host="host.docker.internal",
    )
    cursor = connection.cursor()

    postgres_insert_query = """ INSERT into employee(name, state) VALUES (%s, %s)"""
    record_to_insert = ("AA", "BB")
    sql = f"INSERT INTO items (embedding) VALUES ('{vector}');"
    cursor.execute(sql)

    connection.commit()
    count = cursor.rowcount
    print(count, "Record inserted successfully into mobile table")

except (Exception, psycopg2.Error) as error:
    print("Failed to insert record into mobile table", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

Failed to insert record into mobile table no results to fetch
PostgreSQL connection is closed


In [14]:
query = "What did the president say about Russia"
similar = db.similarity_search_with_score(query, k=2)

for doc in similar:
    print(doc, end="\n\n")

NameError: name 'db' is not defined

In [ ]:
try:
    connection = psycopg2.connect(
        database="postgres",
        user="postgres",
        password="postgres",
        host="host.docker.internal",
    )
    cursor = connection.cursor()

    postgres_insert_query = """ INSERT into employee(name, state) VALUES (%s, %s)"""
    record_to_insert = ("ZZ", "YY")
    cursor.execute(postgres_insert_query, record_to_insert)

    connection.commit()
    count = cursor.rowcount
    print(count, "Record inserted successfully into mobile table")

except (Exception, psycopg2.Error) as error:
    print("Failed to insert record into mobile table", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")